In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
%run /Workspace/FMCG_Project/01_setup/02_schema_utilities

In [0]:
dbutils.widgets.text("catalog", "fmcg", "catalog")
dbutils.widgets.text("data_source", "orders", "data_source")

catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

print(f"{catalog} - {data_source}")

fmcg - orders


In [0]:
base_path = f"/Volumes/fmcg/bronze/souce_data/chaild_company/full_load/orders"
landing_path = f"{base_path}/landing/"
processed_path = f"{base_path}/processed/"

print(base_path)
print(landing_path)
print(processed_path)

/Volumes/fmcg/bronze/souce_data/chaild_company/full_load/orders
/Volumes/fmcg/bronze/souce_data/chaild_company/full_load/orders/landing/
/Volumes/fmcg/bronze/souce_data/chaild_company/full_load/orders/processed/


## Incremental bronze layer 

In [0]:
df_bronze = spark.read.format("csv")\
                    .option("inferSchema", True)\
                    .option("header", True)\
                    .load(f"{landing_path}*csv")\
                    .withColumn("read_timestamp", F.current_timestamp())\
                    .select("*", "_metadata.file_name", "_metadata.file_size")

In [0]:
df_bronze.write.format("delta")\
                .mode("append")\
                .option("enableChangeDataFeed", True)\
                .option("mergeSchema", True)\
                .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

In [0]:
df_bronze.write.format("delta")\
                .mode("overwrite")\
                .option("enableChangeDataFeed", True)\
                .option("mergeSchema", True)\
                .saveAsTable(f"{catalog}.{bronze_schema}.stagging_{data_source}")

In [0]:
dbutils.fs.mv(landing_path,processed_path, recurse=True)

True

## Silver layer

In [0]:
df_silver = spark.read.table(f"{catalog}.{bronze_schema}.stagging_{data_source}")

display(df_silver.limit(10))

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size
FDEC83401502,"Tuesday, December 02, 2025",789401,25891203,256.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC83401502,"Tuesday, December 02, 2025",789401,25891502,218.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC83401502,"Tuesday, December 02, 2025",789401,25891403,280.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC83401502,"Tuesday, December 02, 2025",789401,25891201,262.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC83401502,"Tuesday, December 02, 2025",789401,25891203,256.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC83401502,"Tuesday, December 02, 2025",789401,25891403,280.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC84202603,"Tuesday, December 02, 2025",789202,25891502,218.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC84202603,2025/12/02,789202,25891403,267.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC84202603,"Tuesday, December 02, 2025",789202,25891601,69.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621
FDEC84202603,"Tuesday, December 02, 2025",789202,25891602,126.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621


In [0]:
# Keep only rows with non-null order_qty
df_silver = df_silver.filter(F.col("order_qty").isNotNull())

# Keep only number & valid customer id
df_silver = df_silver.withColumn(
                        "customer_id",
                        F.when(F.col("customer_id").rlike("^[0-9]+$"), F.col("customer_id"))
                        .otherwise(F.lit(99999)).cast("string")
                    )

In [0]:
df_silver = df_silver.withColumn("order_placement_date", F.regexp_replace(F.col("order_placement_date"), r"^[A-Za-z]+,\s*", ""))

In [0]:

df_silver = df_silver.withColumn(
            "order_placement_date",
            F.coalesce(
                F.try_to_date(F.col("order_placement_date"), "yyyy/MM/dd"),
                F.try_to_date(F.col("order_placement_date"), "dd/MM/yyyy"),
                F.try_to_date(F.col("order_placement_date"), "yyyy-MM-dd"),
                F.try_to_date(F.col("order_placement_date"), "dd-MM-yyyy"),
                F.try_to_date(F.col("order_placement_date"), "MMMM dd, yyyy"),
            )
        )

In [0]:
df_silver = df_silver.dropDuplicates(subset=["order_id", "order_placement_date", "customer_id", "order_id", "order_qty"])

df_silver = df_silver.withColumn("product_id", F.col("product_id").cast("string"))

In [0]:
df_products = spark.read.table(f"{catalog}.{silver_schema}.products")

df_joined = df_silver.join(df_products.select("product_id", "product_code"), on="product_id", how="inner").select(df_silver["*"], df_products["product_code"])

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size,product_code
FDEC85902401,2025-12-02,789902,25891401,339.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,da6bfc596c1360ca07bda4e0ae6bfe3b8456517fc6e8ddc265630ff940f9ab05
FDEC84702602,2025-12-03,789702,25891602,143.0,2025-11-30T17:39:11.751Z,orders_2025_12_03.csv,21899,778c2a7aa27bfdb211fd5ece048de80d00fbf3d6924bd908d91054796ba16ab6
FDEC83221503,2025-12-02,789221,25891202,490.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,0cb7b2f42657b625f754e833aa1cf6a967be26f17415f5342302ebb0e90c8a28
FDEC84520503,2025-12-02,789520,25891503,132.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,062f5574bbdf4386b2c7c6075483b417b4a00b172fcba919dbba7dae1b774379
FDEC83201403,2025-12-02,789201,25891403,363.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9
FDEC86420303,2025-12-03,789420,25891201,146.0,2025-11-30T17:39:11.751Z,orders_2025_12_03.csv,21899,2e387cef1424d6e7b162b45622d4b1a788d11776e33d05cc8552f4ecd2ea1896
FDEC84521503,2025-12-02,789521,25891503,162.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,062f5574bbdf4386b2c7c6075483b417b4a00b172fcba919dbba7dae1b774379
FDEC83720103,2025-12-02,789720,25891103,338.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268
FDEC84202603,2025-12-02,789202,25891403,267.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9
FDEC85703302,2025-12-02,789703,25891302,37.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,d9ebd1ca64d23951a6310af93b1c5ac27d831ac842e89aea59a9e8b38621faa5


In [0]:
if not (spark.catalog.tableExists(f"{catalog}.{silver_schema}.{data_source}")):
    df_joined.write.format("delta")\
                    .mode("overwrite")\
                    .option("mergeSchema", True)\
                    .option("enableChangeDataFeed", True)\
                    .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")
else:
    delta_trg_table = DeltaTable.forName(spark, f"{catalog}.{silver_schema}.{data_source}")

    delta_trg_table.alias("trg").merge(
        df_joined.alias("src"),
        condition="trg.order_placement_date=src.order_placement_date AND trg.order_id=src.order_id AND trg.product_code=src.product_code AND trg.customer_id=src.customer_id"
    ).whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

In [0]:
df_joined.write.format("delta")\
                    .mode("overwrite")\
                    .option("mergeSchema", True)\
                    .option("enableChangeDataFeed", True)\
                    .saveAsTable(f"{catalog}.{silver_schema}.stagging_{data_source}")

## Gold layer

In [0]:
df_gold = spark.read.table(f"{catalog}.{silver_schema}.stagging_{data_source}")
display(df_gold.limit(10))

order_id,order_placement_date,customer_id,product_id,order_qty,read_timestamp,file_name,file_size,product_code
FDEC85902401,2025-12-02,789902,25891401,339.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,da6bfc596c1360ca07bda4e0ae6bfe3b8456517fc6e8ddc265630ff940f9ab05
FDEC84702602,2025-12-03,789702,25891602,143.0,2025-11-30T17:39:11.751Z,orders_2025_12_03.csv,21899,778c2a7aa27bfdb211fd5ece048de80d00fbf3d6924bd908d91054796ba16ab6
FDEC83221503,2025-12-02,789221,25891202,490.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,0cb7b2f42657b625f754e833aa1cf6a967be26f17415f5342302ebb0e90c8a28
FDEC84520503,2025-12-02,789520,25891503,132.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,062f5574bbdf4386b2c7c6075483b417b4a00b172fcba919dbba7dae1b774379
FDEC83201403,2025-12-02,789201,25891403,363.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9
FDEC86420303,2025-12-03,789420,25891201,146.0,2025-11-30T17:39:11.751Z,orders_2025_12_03.csv,21899,2e387cef1424d6e7b162b45622d4b1a788d11776e33d05cc8552f4ecd2ea1896
FDEC84521503,2025-12-02,789521,25891503,162.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,062f5574bbdf4386b2c7c6075483b417b4a00b172fcba919dbba7dae1b774379
FDEC83720103,2025-12-02,789720,25891103,338.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,102628255d24304d6bbe0438b1ac992054f262e0814d306d0a34d7356cef3268
FDEC84202603,2025-12-02,789202,25891403,267.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,77b6f538a9d0e0cf845db5c2cbecec46fdd30303b501e06f64baf1d4dc0e66f9
FDEC85703302,2025-12-02,789703,25891302,37.0,2025-11-30T17:39:11.751Z,orders_2025_12_02.csv,19621,d9ebd1ca64d23951a6310af93b1c5ac27d831ac842e89aea59a9e8b38621faa5


In [0]:
df_gold = df_gold.selectExpr("order_id", "order_placement_date AS date", "customer_id AS customer_code", "product_code", "product_id", "order_qty AS sold_quantity")

In [0]:
if not (spark.catalog.tableExists(f"{catalog}.{gold_schema}.sb_fact_{data_source}")):
    df_gold.write.format("delta")\
                    .mode("overwrite")\
                    .option("mergeSchema", True)\
                    .option("enableChangeDataFeed", True)\
                    .saveAsTable(f"{catalog}.{gold_schema}.sb_fact_{data_source}")
else:
    delta_trg_table = DeltaTable.forName(spark, f"{catalog}.{gold_schema}.sb_fact_{data_source}")

    delta_trg_table.alias("trg").merge(
        df_gold.alias("src"),
        condition="trg.date=src.date AND trg.order_id=src.order_id AND trg.product_code=src.product_code AND trg.customer_code=src.customer_code"
    ).whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()\
    .execute()

### merge with parent compmay

In [0]:
df_src = spark.read.table(f"{catalog}.{silver_schema}.stagging_{data_source}")

df_src = df_src.selectExpr("order_placement_date AS date", "product_code", "customer_id AS customer_code", "order_qty AS sold_quantity")

df_src = df_src.withColumn("date", F.trunc(F.col("date"), "MM"))

df_src = df_src.groupBy("date", "product_code", "customer_code").agg(F.sum(F.col("sold_quantity")).alias("sold_quantity"))

In [0]:
delta_trg_table = DeltaTable.forName(spark, f"{catalog}.{gold_schema}.fact_{data_source}")

delta_trg_table.alias("trg").merge(
    df_src.alias("src"),
    condition="trg.date=src.date AND trg.product_code=src.product_code AND trg.customer_code=src.customer_code"
).whenMatchedUpdateAll()\
.whenNotMatchedInsertAll()\
.execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]